<a href="https://colab.research.google.com/github/BraedenCu/ObjectDetectionWithPytorchTutorial/blob/master/vexaiPytorch4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessary packages (should be automatically installed in colab)
If you get an import error, install each of them using python3 -m pip install ____

Next we clone my fork of the pytorch vision repository, which contains scripts we will use later. Then clone some useful scripts for data manipluation from my repository. Finally, create directories for annotations and images.

In [ ]:
#EVERYTIME KERNEL DIES, YOU NEED TO REUPLOAD XMLS AND IMAGES

import torch
import torchvision
import pycocotools
import os

%cd /root/
#need to clone my fork, elsewise it will throw import error when using one of the libraries
#remove and previous installation, clone the most recent
!rm -r vision
!git clone https://github.com/BraedenCu/vision.git
!cd vision
!git checkout v0.6.0

%cd ~
#remove any previous clone of the repo and install the latest release 
!rm -r scriptsForObjectDetectionDataPreparation/
!git clone https://github.com/BraedenCu/scriptsForObjectDetectionDataPreparation.git
!mkdir annotations
!mkdir images

Here we are first removing any prevois copys of the following scripts and updating them. These scripts will be used later for training.


In [ ]:
#in order to gain access to functions defined in engine.py/utils.py/etc. we need to copy
#the python scripts to root
%cd /root/
#remove old versions of the python scripts and replace with updated ones
!rm -r engine.py
!rm -r coco_eval.py
!rm -r coco_utils.py
!rm -r utils.py
%cd vision
%cd references
%cd detection
#copy files to root
!cp engine.py -b /root/
!cp utils.py -b /root/
!cp coco_utils.py -b /root/
!cp coco_eval.py -b /root/
#change directory back to root
%cd /root/
!ls

Convert to csv

In [ ]:
%cd ~/scriptsForObjectDetectionDataPreparation/
#convert xml_to_csv, if it failes to create do to empty csv, be sure to check that you have uploaded your images and annottions
#remove and reupdate
!rm -r vex_labels.csv
!python xml_to_csv.py

In [ ]:
import os
import numpy as np
import torch
from PIL import Image
import pandas as pd

#function to return data when given filename
def parse_one_annot(path_to_csv_file, filename):
   data = pd.read_csv(path_to_csv_file)
   boxes_array = data[data["filename"] == filename][["xmin", "ymin",        
   "xmax", "ymax"]].values
   #print(boxes_array)
   return boxes_array

   
class vex(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = "/root/images" #sorted(os.listdir(os.path.join(root, "images")))
        self.csv_path = "/root/scriptsForObjectDetectionDataPreparation/vex_labels.csv"

    def __getitem__(self, idx):
        # load images ad masks
        #simply just repeat 1 rather than search for 0.jpeg, which does not exist
        if idx == 0:
          idx = 1
        
        img_path = "/root/images/{}{}".format(idx, ".jpeg")
        #print(img_path)
        img = Image.open(img_path).convert("RGB")
        
        # convert the PIL Image into a numpy array
        #box_list = parse_one_annot(self.csv_path, self.imgs[idx])
        box_list = parse_one_annot(self.csv_path, img_path)
        boxes = torch.as_tensor(box_list, dtype=torch.float32)
        num_objs = len(box_list)
        # 5 classes, so five boxes to be either zero or one
        num_objs = 5
        labels = torch.ones((num_objs), dtype=torch.int64)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:,0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
    
        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        #target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        #if self.transforms is not None:
        img = self.transforms(img)
            #target = self.transforms(img)
        return img, target

    def __len__(self):
        #return len(self.imgs)
        return 204

Below just checks to make sure your image path is correct.
I spent alot of time smashing my head on my keyboard attempting to solve meaningless errors when it was simply due to an incorrect image path. Don't be me, check to make sure the path is correct. After you run this code you must rerun the above excerpt. (then comment it back out)


In [ ]:
#makes sure image path is correct
#img_path = "/root/images/{}{}".format(idx, ".jpeg")
#print(img_path)
#img = Image.open(img_path).convert("RGB")

Load the model

In [ ]:
def get_model_instance_segmentation(num_classes):
   # load an object detection model pre-trained on COCO
   model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
  # get the number of input features for the classifier
   in_features = model.roi_heads.box_predictor.cls_score.in_features
   # replace the pre-trained head with a new on
   model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
   
   return model


Need to convert all images to tensors

In [ ]:
from torchvision import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

Below is simply a test to make sure the data pipepile is functional, you do not need to run this

In [ ]:
import utils

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
dataset = vex('/root/', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
 dataset, batch_size=2, shuffle=True, num_workers=4,
 collate_fn=utils.collate_fn)

# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets) 

# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions

Change directory to root
Then train the model

In [ ]:
%cd /root

/root


In [ ]:
from engine import train_one_epoch, evaluate
import utils

def main():
    # train on the GPU or on the CPU, if a GPU is not available
    #Running in GPU will GREATLY INCREASE training time, SO DO IT!!!!
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    #My dataset was utilizing 5 classes, change to your number of classes
    num_classes = 5
    # use our dataset and defined transformations
    dataset = vex('/root/', get_transform(train=True))
    dataset_test = vex('/root/', get_transform(train=False))

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=4,
        collate_fn=utils.collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn)

    #get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    #move model to the right device (cpu or gpu if possible)
    model.to(device)

    #construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    #and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    #train it for 5 epochs
    num_epochs = 5

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

    print("training complete!")

main()


Save the model


In [ ]:
#save model to training/models
#remove prior model saves, comment out the following line if you DO NOT want to delete past models
!rm -r training
!mkdir training
torch.save(model.state_dict(), "/root/training/model")

Test out your newly saved model on an image from your dataset. 

In [ ]:
from PIL import ImageDraw

#get saved model
loaded_model = get_model_instance_segmentation(num_classes = 5)
loaded_model.load_state_dict(torch.load("/root/training/model"))
#create new instance of dataset to access images from
dataset_test = vex('/root/', get_transform(train=False))
#image number
idx = 0
img, _ = dataset_test[idx]
#get the correct labels
label_boxes = np.array(dataset_test[idx][1]["boxes"])
#put the model in evaluation mode
loaded_model.eval()
with torch.no_grad():
   prediction = loaded_model([img])
image = Image.fromarray(img.mul(255).permute(1, 2,0).byte().numpy())
draw = ImageDraw.Draw(image)
#draws correct labels
for elem in range(len(label_boxes)):
   draw.rectangle([(label_boxes[elem][0], label_boxes[elem][1]),
   (label_boxes[elem][2], label_boxes[elem][3])], 
   outline ="green", width =3)
#make predictions
for element in range(len(prediction[0]["boxes"])):
   boxes = prediction[0]["boxes"][element].cpu().numpy()
   score = np.round(prediction[0]["scores"][element].cpu().numpy(),
                    decimals= 4)
   #change this score to modify the value at which the boxes will be drawn. For example, the default threshold is 0.45, so the model must have atleast 
   #45% confidence in the location of the box in order to draw it on the image.
   if score > 0.45:
      draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])], 
      outline ="red", width =3)
      draw.text((boxes[0], boxes[1]), text = str(score))
#display the image
image